In [ ]:
from diart.sources import MicrophoneAudioSource
from diart.inference import RealTimeInference
from diart.pipelines import OnlineSpeakerDiarization, PipelineConfig

pipeline = OnlineSpeakerDiarization(PipelineConfig())
audio_source = MicrophoneAudioSource(pipeline.sample_rate)
inference = RealTimeInference("C:\Users\User\Desktop\HDK_JPN", do_plot=True)

inference(pipeline, audio_source)

In [2]:
import rx
import rx.operators as ops
import diart.operators as dops
from diart.sources import MicrophoneAudioSource
from diart.blocks import FramewiseModel, OverlapAwareSpeakerEmbedding

sample_rate = 16000
mic = MicrophoneAudioSource(sample_rate)

# Initialize independent modules
segmentation = FramewiseModel("pyannote/segmentation")
embedding = OverlapAwareSpeakerEmbedding("pyannote/embedding")

# Reformat microphone stream. Defaults to 5s duration and 500ms shift
regular_stream = mic.stream.pipe(dops.regularize_stream(sample_rate))
# Branch the microphone stream to calculate segmentation
segmentation_stream = regular_stream.pipe(ops.map(segmentation))
# Join audio and segmentation stream to calculate speaker embeddings
embedding_stream = rx.zip(
    regular_stream, segmentation_stream
).pipe(ops.starmap(embedding))

embedding_stream.subscribe(on_next=lambda emb: print(emb.shape))

mic.read()

KeyboardInterrupt: 

In [4]:
!python -m diart.stream microphone

^C
